In [13]:
import pandas as pd
import numpy as np
train_df=pd.read_csv(r'D:\Dataset\Plant pathology\train.csv')

In [14]:
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [15]:
labels=train_df.iloc[:,1:5].values

In [16]:
train_y = labels[:, 2] + labels[:, 3] * 2 + labels[:, 1] * 3
train_df['target']=train_y

In [17]:
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab,target
0,Train_0,0,0,0,1,2
1,Train_1,0,1,0,0,3
2,Train_2,1,0,0,0,0
3,Train_3,0,0,1,0,1
4,Train_4,1,0,0,0,0


In [12]:
import os
import pandas as pd
from sklearn import model_selection

if __name__ == "__main__":
    input_path = "D:\\Dataset\\Plant pathology\\"
    df=train_df.copy()
    df["kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    y = df.target.values
    skf = model_selection.StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    for fold,(idxT,idxV) in enumerate(skf.split(X=df, y=y)):
        df.loc[idxV, "kfold"] = fold
    print(df.target.value_counts())
    df.to_csv(os.path.join(input_path, "train_folds.csv"), index=False)

1    622
2    592
0    516
3     91
Name: target, dtype: int64


In [1]:
from PIL import Image
import torch
import numpy as np

class ClassificationDataset:
    
    def __init__(self, image_paths, targets, resize, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image),
            "targets": torch.tensor(targets),
        }

In [1]:
import torch.nn as nn

class DenseCrossEntropy(nn.Module):

    def __init__(self):
        super(DenseCrossEntropy, self).__init__()
        
        
    def forward(self, logits, labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits, dim=-1)
        
        loss = -labels * logprobs
        loss = loss.sum(-1)

        return loss.mean()

In [2]:
import torch
import torchvision

class DenseNet(nn.Module):

    def __init__(self):
        super(DenseNet, self).__init__()
        
        self.base_model = torchvision.models.densenet201(pretrained=True)
        self.base_model.classifier = nn.Identity()
        self.fc = torch.nn.Sequential(
                    torch.nn.Linear(1920, 1024, bias = True),
                    torch.nn.BatchNorm1d(1024),
                    torch.nn.ReLU(inplace=True),
                    torch.nn.Dropout(0.3),
                    torch.nn.Linear(1024, 512, bias = True),
                    torch.nn.BatchNorm1d(512),
                    torch.nn.ReLU(inplace=True),
                    torch.nn.Dropout(0.3),
                    torch.nn.Linear(512, 4))
        
    def forward(self, image, targets):
        x = self.base_model(image)
        out = self.fc(x)
        
        loss=None
        
        if targets is not None:
            loss = DenseCrossEntropy()(out, targets.type_as(out))
        
        return out, loss

In [3]:
import torch.nn as nn
import efficientnet_pytorch


"""
class PlantModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.backbone = torchvision.models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features

        self.out = nn.Linear(in_features, 4)
        
    def forward(self, image, targets=None):
        batch_size, C, H, W = image.shape
        
        x = self.backbone.conv1(image)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.25, self.training)

        x = self.out(x)
        
        loss=None
        
        if targets is not None:
            loss = DenseCrossEntropy()(x, targets.type_as(x))
            
        return x, loss
"""

class EfficientNet(nn.Module):
    def __init__(self):
        super().__init__()        

        self.features = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b5')
        #print(self.features)
        self.classification = nn.Linear(2048, 4)
        #print(self.features)

        
        
    def forward(self, image, targets):    
        
        image = self.features.extract_features(image)
            
        image = F.avg_pool2d(image, image.size()[2:]).reshape(-1, 2048)
        
        out = self.classification(image)
        
        loss=None
        
        if targets is not None:
            loss = DenseCrossEntropy()(out, targets.type_as(out))
        
        return out, loss

In [5]:
import os
import torch
import torchvision
import pandas as pd
from sklearn import model_selection
import albumentations 
import pretrainedmodels
import albumentations as A
import albumentations as aug
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import KFold
import PlantDataset

from apex import amp
from sklearn import metrics
from torch.nn import functional as F

from wtfml.data_loaders.image import ClassificationLoader
from wtfml.engine import Engine
from wtfml.utils import EarlyStopping
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)



def train(fold):
    training_data_path = "D:\\Dataset\\Plant pathology\\images\\"
    model_path = "D:\\Dataset\\Plant pathology\\"
    df = pd.read_csv("D:\\Dataset\\Plant pathology\\train_folds.csv")
    device = "cuda"
    epochs = 100
    train_bs = 4
    valid_bs = 4
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    p=0.5
    SIZE=512
    
    df_train=df.loc[df.kfold!=0].reset_index(drop=True)
    df_valid=df.loc[df.kfold==0].reset_index(drop=True)
    
    train_aug = albumentations.Compose(
        [
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),

    A.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),  
        ]
   
    )

    valid_aug = albumentations.Compose(
        [
            A.Resize(height=SIZE, width=SIZE, p=1.0),
            A.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        ]
    )

    train_images = df_train.image_id.values.tolist()
    train_images = [os.path.join(training_data_path, i + ".jpg") for i in train_images]
    train_targets = df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].values
    #print(len(train_images))
    #print(train_targets.shape)

    valid_images = df_valid.image_id.values.tolist()
    valid_images = [os.path.join(training_data_path, i + ".jpg") for i in valid_images]
    valid_targets = df_valid[['healthy', 'multiple_diseases', 'rust', 'scab']].values
    #print(len(valid_images))
    #print(valid_targets.shape)

    train_dataset = PlantDataset.ClassificationDataset(
        image_paths=train_images,
        targets=train_targets,
        resize=(512,512),
        augmentations=train_aug
    )
    

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=train_bs,
        shuffle=True,
        num_workers=4
    )
    


    valid_dataset = PlantDataset.ClassificationDataset(
        image_paths=valid_images,
        targets=valid_targets,
        resize=(512,512),
        augmentations=valid_aug
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=valid_bs,
        shuffle=False,
        num_workers=4)

    model = DenseNet()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        mode="max"
    )

    model, optimizer = amp.initialize(
        model,
        optimizer,
        opt_level="O1",
        verbosity=0
    )

    es = EarlyStopping(patience=7, mode="max")
    for epoch in range(epochs):
        training_loss = Engine.train(
            train_loader, 
            model,
            optimizer,
            device,
            fp16=True
        )
        predictions, valid_loss = Engine.evaluate(
            valid_loader, 
            model,
            device
        )
        predictions = np.concatenate(predictions,axis=0)
        #print(predictions)
        #print(predictions.shape)
        #print(valid_targets.shape)
        auc = metrics.roc_auc_score(valid_targets, predictions, average='macro')
        scheduler.step(auc)
        print(f"epoch={epoch}, auc={auc}")
        es(auc, model, os.path.join(model_path, f"model{fold}.bin"))
        if es.early_stop:
            print("early stopping")
            break


def predict(fold):
    test_data_path = "D:\\Dataset\\Plant pathology\\images\\"
    model_path = "D:\\Dataset\\Plant pathology\\"
    df_test = pd.read_csv("D:\\Dataset\\Plant pathology\\test.csv")
    df_test.loc[:, "target"] = 0
    p=0.5
    device = "cuda"
    epochs = 100
    test_bs = 32
    SIZE=512
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    df_test['healthy']=-1
    df_test['multiple_diseases']=-1
    df_test['rust']=-1
    df_test['scab']=-1
    
    train_aug = albumentations.Compose(
        [
            
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),
    A.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        ]
    )

    valid_aug = albumentations.Compose(
        [
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        ]
    )


    test_images = df_test.image_id.values.tolist()
    test_images = [os.path.join(test_data_path, i + ".jpg") for i in test_images]
    test_targets =df_test[['healthy', 'multiple_diseases', 'rust', 'scab']].values

    
    test_dataset = ClassificationLoader(
        image_paths=test_images,
        targets=test_targets,
        resize=(512,512),
        augmentations=train_aug
    )
    #print(len(test_dataset))
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=test_bs,
        shuffle=False,
        num_workers=4
    )

    model = DenseNet()
    model.load_state_dict(torch.load(os.path.join(model_path, f"model{fold}.bin")))
    model.to(device)

    predictions = Engine.predict(
        test_loader,
        model,
        device
    )
    return np.concatenate(predictions,axis=0)


if __name__ == "__main__":
    pred1=TTA_Wrapper(15,(1821,4),fold=0)
    pred2=TTA_Wrapper(15,(1821,4),fold=1)
    pred3=TTA_Wrapper(15,(1821,4),fold=2)
    pred4=TTA_Wrapper(15,(1821,4),fold=3)
    pred5=TTA_Wrapper(15,(1821,4),fold=4)

100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:52<00:00,  1.97s/it]


TTA 0


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:52<00:00,  1.97s/it]


TTA 1


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:55<00:00,  2.02s/it]


...

TTA 12


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [02:01<00:00,  2.13s/it]


TTA 13


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [02:01<00:00,  2.13s/it]

TTA 14


In [2]:
    pred1=TTA_Wrapper(15,(1821,4),fold=0)
    pred2=TTA_Wrapper(15,(1821,4),fold=1)
    pred3=TTA_Wrapper(15,(1821,4),fold=2)
    pred4=TTA_Wrapper(15,(1821,4),fold=3)
    pred5=TTA_Wrapper(15,(1821,4),fold=4)

[[ 0  1  3]
 [ 5  7  9]
 [ 0  2  4]
 [ 6  8 10]]


In [7]:
pred1

array([[-0.38189757, -0.31893981,  1.41436013, -0.82367279],
       [-0.37767892, -0.57825316,  1.50319936, -0.5525447 ],
       [-0.66295427, -1.4600713 , -2.29893449,  3.61607366],
       ...,
       [-0.91172348, -0.66970113,  2.72168687, -1.52579848],
       [ 3.13425418, -1.92370196, -0.53892936, -0.54153712],
       [-0.44715526, -0.71655816, -0.8408108 ,  2.07446277]])

In [8]:
final_sub=(pred1+pred2+pred3+pred4+pred5)/5

In [9]:
final_sub

array([[-0.58887086, -0.39831735,  1.69062382, -0.83030524],
       [-0.69912932, -0.56662958,  1.84701244, -0.67613126],
       [-0.35647314, -1.38374281, -1.82156709,  3.32016097],
       ...,
       [-0.85465569, -0.79984985,  2.76028034, -1.50560158],
       [ 3.42605112, -2.10936812, -0.25774029, -1.20440306],
       [-0.73759788, -0.3644933 , -0.87687612,  1.93343454]])

In [13]:
submission=pd.read_csv('D:\\Dataset\\Plant pathology\\sample_submission.csv')

In [14]:
submission.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [4]:
def TTA_Wrapper(tta, rec, fold):  
    preds=np.zeros((rec))
    for tta_id in range(tta):
        prediction=predict(fold=fold)
        preds += np.array(prediction)
        print(f'TTA {tta_id}')
    final=(preds / tta)
    return final

In [12]:
submission[['healthy', 'multiple_diseases', 'rust', 'scab']] = torch.softmax(torch.tensor(final_sub,dtype=torch.float), dim=1)
submission.to_csv('D:\\Dataset\\Plant pathology\\submission_512_512_densenet_v1.csv', index=False)

In [15]:
subhead1 = pd.read_csv("D:\\Dataset\\Plant pathology\\submission_512_512_tta.csv")
subhead2 = pd.read_csv("D:\\Dataset\\Plant pathology\\submission_2.csv")
subhead3 = pd.read_csv("D:\\Dataset\\Plant pathology\\submission_256_256_densenet_v1.csv")
subhead4 = pd.read_csv("D:\\Dataset\\Plant pathology\\submission_512_512_densenet_v1.csv")

In [16]:
submission[['healthy', 'multiple_diseases', 'rust', 'scab']]=(subhead1[['healthy','multiple_diseases','rust','scab']]+subhead3[['healthy','multiple_diseases','rust','scab']]+subhead4[['healthy','multiple_diseases','rust','scab']])/3

In [17]:
submission.to_csv('D:\\Dataset\\Plant pathology\\submission_models_v2.csv', index=False)

In [16]:
submission.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.006439,0.053717,0.936657,0.003187
1,Test_1,0.002860,0.014780,0.980345,0.002015
2,Test_2,0.002308,0.001740,0.000283,0.995670
3,Test_3,0.985029,0.001098,0.010911,0.002962
4,Test_4,0.000217,0.001690,0.997611,0.000482


In [28]:
(subhead1[['healthy','multiple_diseases','rust','scab']]+subhead2[['healthy','multiple_diseases','rust','scab']])/2

,healthy,multiple_diseases,rust,scab
0,0.000671,0.042982,0.956228,1.200669e-04
1,0.000566,0.016645,0.982528,2.613488e-04
2,0.000161,0.001041,0.000018,9.987799e-01
3,0.989236,0.000480,0.009266,1.017837e-03
4,0.000002,0.000110,0.999887,7.345844e-07
...,...,...,...,...
1816,0.000297,0.005113,0.994514,7.632535e-05
1817,0.031991,0.735780,0.040193,1.920368e-01
1818,0.000180,0.006243,0.993521,5.540416e-05
1819,0.992510,0.000991,0.003900,2.599078e-03
